**1857. Largest Color Value in a Directed Graph**

https://leetcode.com/problems/largest-color-value-in-a-directed-graph/

In [54]:
# times out
def largestPathValue(colors: str, edges: list[list[int]]) -> int:
    if not edges:
        return 1
    
    graph = {}
    from_nodes = set()
    to_nodes = set()
    for k, v in edges:
        from_nodes.add(k)
        to_nodes.add(v)
        if k in graph:
            graph[k].append(v)
        else:
            graph[k] = [v]

    from_nodes -= to_nodes

    if not from_nodes:
        return -1
    
    largest_color_value = 0

    def bfs(start):
        s = [[start, dict(), set()]]
        while s:
            i, color_dic, visited = s.pop()
            visited.add(i)
            color_dic[colors[i]] = color_dic.get(colors[i], 0) + 1
            nonlocal largest_color_value
            largest_color_value = max(largest_color_value, color_dic[colors[i]])
            if i in graph:
                for n in graph[i]:
                    if n in visited:
                        return False
                    else:
                        s.append([n, color_dic.copy(), visited.copy()])
        return True

    for node in from_nodes:
        if not bfs(node):
            return -1

    return largest_color_value

print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4]])) # 3
print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4],[4,1]])) # 3
print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4],[4,2]])) # -1
print(largestPathValue(colors = "bbbhb", edges = [[0,2],[3,0],[1,3],[4,1]])) # 4
print(largestPathValue(colors = "a", edges = [[0,0]])) # -1
print(largestPathValue(colors = "g", edges = [])) # -1

3
3
-1
4
-1
1


In [68]:
# uses topological sort
def largestPathValue(colors: str, edges: list[list[int]]) -> int:

    # no edges = max path is 1
    if not edges:
        return 1
    
    digraph = {}
    indegrees = {}

    for k, v in edges:
        if k in digraph:
            digraph[k].append(v)
        else:
            digraph[k] = [v]
        # this step is needed to store nodes with no outcoming edges
        if v not in digraph:
            digraph[v] = []
        # counting indegrees
        indegrees[v] = indegrees.get(v, 0) + 1

    topological_sort = []
    no_incoming_edges = list(digraph.keys() - indegrees.keys())

    # populating topological sort
    while no_incoming_edges:
        node = no_incoming_edges.pop()
        topological_sort.append(node)
        for n in digraph[node]:
            indegrees[n] -= 1
            if indegrees[n] == 0:
                no_incoming_edges.append(n)

    # this means there is a loop
    if len(digraph) != len(topological_sort):
        return -1
    
    largest_color_value = 0

    # maintaining a color dictionary for each node
    colour_dicts = {k:dict() for k in digraph.keys()}

    # going backwards in the topological sort
    for i in range(len(topological_sort)-1, -1, -1):
        index = topological_sort[i]

        # if node has outgoing edges
        if digraph[index]:
            for n in digraph[index]:
                for color in colour_dicts[n]:
                    # choosing maximum possible number of colours available to any child
                    colour_dicts[index][color] = max(colour_dicts[index].get(color, 0), colour_dicts[n][color])
        # incrementing the node colour counter
        colour_dicts[index][colors[index]] = colour_dicts[index].get(colors[index], 0) + 1
        # potentially updating the largest_color_value counter
        largest_color_value = max(largest_color_value, colour_dicts[index][colors[index]])

    return largest_color_value


print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4]])) # 3
print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4],[4,1]])) # 3
print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4],[4,2]])) # -1
print(largestPathValue(colors = "bbbhb", edges = [[0,2],[3,0],[1,3],[4,1]])) # 4
print(largestPathValue(colors = "a", edges = [[0,0]])) # -1
print(largestPathValue(colors = "g", edges = [])) # -1

3
3
-1
4
-1
1
